In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

2022-10-28 17:08:30.723451: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-28 17:08:30.911014: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-28 17:08:31.533318: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib.real:/opt/conda/lib/python3.8/site-packages/torch/lib:/opt/conda/lib/python3.8/site-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64


['/device:CPU:0', '/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7']


2022-10-28 17:08:32.455979: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-28 17:08:36.968878: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 38407 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2022-10-28 17:08:36.970766: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 38407 MB memory:  -> device: 1, name: A100-SXM4-40GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
2022-10-28 17:08:36.972595: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:2 with 38407 MB memory:  -> device: 2, name: A100-SXM4-40GB, pci bus id: 0000:47:00.0, compute capab

In [4]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [5]:
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [6]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [7]:
import nltk
nltk.download('all');

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

In [8]:
import pandas as pd
import numpy as np
from transformers import pipeline
from tqdm import tqdm
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import time
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import codecs
from nltk.tokenize import sent_tokenize

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading data set

In [9]:
%cd ..

/workspace/asurion_f22_muby


In [10]:
transcript = pd.read_csv("fe_data_new/total_collated_transcripts.csv")

In [11]:
transcript["encrypted_collated_transcription"] = transcript["encrypted_collated_transcription"].astype(str)

In [12]:
transcript_1000 = transcript.iloc[:1000,:]

In [13]:
from transformers import BertForSequenceClassification, BertTokenizer

# Define the model repo
myckpt = 'bert-base-cased'
mytokenizer = BertTokenizer.from_pretrained("bert-base-cased", is_split_into_words=True)
device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu") 

# Download pytorch model
#model = BertForSequenceClassification.from_pretrained(myckpt).to(device)

In [14]:
sentence = list(transcript_1000.iloc[:,3])
#sentence_token_nltk(sentence_1000[0])

In [15]:
from transformers import pipeline

In [16]:
def transcript_chunking(sentence):
    length = 512
    sent_list = sent_tokenize(sentence)
    count = 0
    token_list, input_list = [], []
    #mask_list, attention_list = [], []
    for sent in sent_list:
            # tokenize sentences
        res = mytokenizer(sent)
        tokens = res['input_ids']
        count += len(tokens)
        if count <= length:
            token_list = token_list + tokens
        else:    
            input_list.append(token_list)
            token_list = tokens
            count=len(tokens)
                
    sentence_chunk = []
    for i in range(len(input_list)):
        tokens = mytokenizer.convert_ids_to_tokens(input_list[i])
        paragraph = mytokenizer.convert_tokens_to_string(tokens)
        paragraph = mytokenizer.clean_up_tokenization(paragraph)
        paragraph = paragraph.replace(" [SEP]","")
        paragraph = paragraph.replace("[CLS] ","")
        paragraph = paragraph.replace(" [SEP] [CLS] ","")
        sentence_chunk.append(paragraph)
    return sentence_chunk

In [17]:
list_of_sentence = []
for i in tqdm(range(len(sentence[:1000]))):
    list_of_sentence.append(transcript_chunking(sentence[i]))

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:55<00:00, 17.86it/s]


In [18]:
len(list_of_sentence)

1000

In [19]:
transcript_1000["chunked"] = list_of_sentence
transcript_1000.head()

/tmp/ipykernel_2040/3950884394.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcript_1000["chunked"] = list_of_sentence


,id,sales_offer_date,accepted_flg,encrypted_collated_transcription,month,chunked
0,e5e4ee3c3c0e9ac72561334e6eb23cc233b1bcd0a0063d...,2022-03-26,False,ok Thank for calling [CLIENT] Tech Hello? Hi. ...,march,[ok Thank for calling [ CLIENT ] Tech Hello? H...
1,ae6128d8d240d28edbcb7eb27256780cf662a2be1aec8c...,2022-03-02,False,"on on Oh, My name is [NAME]. May I please have...",march,"[on on Oh, My name is [ NAME ]. May I please h..."
2,4899f9acdb70b0246b2d0ac2871dbcea8267e57b060154...,2022-03-12,False,Hello. We for calling [CLIENT] Tech Coach. My ...,march,[Hello. We for calling [ CLIENT ] Tech Coach. ...
3,0589751e11caac0eac815faf84f1a366d70b86b78324a2...,2022-03-20,False,Hello. Thank you so much for contacting [CLIEN...,march,[Hello. Thank you so much for contacting [ CLI...
4,6c4790cb02e4f556d73436f79268d5c99de50a9acec40e...,2022-03-15,False,Thank you very much Hello? can I can you hear ...,march,[Thank you very much Hello? can I can you hear...


In [20]:
transcript_1000.to_csv("fe_data_v2/transcript_data_use/mock_transcript.csv", index = False)